In [24]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
import pandas as pd
import time
# Configure Chrome options
chrome_options = Options()
chrome_options.debugger_address = "127.0.0.1:9222"  # connect to the running Chrome


# Attach to the existing window



driver = webdriver.Chrome(options=chrome_options)
page_to_scrape = 100000
# Open 10 empty tabs first
for i in range(9):  # already have 1 tab open by default
    driver.switch_to.new_window("https://phishtank.org/phish_search.php")
    time.sleep(0.5)
    print(driver.window_handles)
print(driver.window_handles)
df = pd.DataFrame({'url':[],'label':[]})
input_ = input()
for page in range(0,page_to_scrape,len(driver.window_handles)):
    for i, handle in enumerate(driver.window_handles):
        driver.switch_to.window(handle)
        driver.get(f"https://phishtank.org/phish_search.php?page={page+1}&valid=y&Search=Search")
        #driver.implicitly_wait(0.1)
        title = driver.title
        print('page number:-',page+1, 'title:-',title)

        dataset = driver.find_element(by=By.CLASS_NAME, value="data")
        #print(dataset.text)
        rows = dataset.find_elements(By.TAG_NAME, "tr")

        for i, row in enumerate(rows):
            # Get all cells in this row
            cells = row.find_elements(By.TAG_NAME, "td")
            data = [cell.text for cell in cells][1:4:2]

            if len(data) == 2 and data[0].count('\n') == 1:
                new_row = pd.Series({'url':data[0].split('\n')[0],'label':data[1]})
                df.loc[len(df)] = new_row
    

'''
text_box = driver.find_element(by=By.NAME, value="my-text")
submit_button = driver.find_element(by=By.CSS_SELECTOR, value="button")
driver.implicitly_wait(10)
text_box.send_keys("Selenium")
submit_button.click()
message = driver.find_element(by=By.ID, value="message")
text = message.text
print(text)
'''
driver.implicitly_wait(10)
driver.quit()

df.to_csv('valid_phishtank_scraped_dataset.csv')


['3384A513DEA3046A943CC75095912B0C', '942206CC5EE6CFEFF38C303A94F7AADD']
['3384A513DEA3046A943CC75095912B0C', '942206CC5EE6CFEFF38C303A94F7AADD', '650DD1BCD2CAF0A4BCA50A3A92F34F53']
['3384A513DEA3046A943CC75095912B0C', '942206CC5EE6CFEFF38C303A94F7AADD', '650DD1BCD2CAF0A4BCA50A3A92F34F53', '6E246C4042A03B3C1BCF84FA2DAE71BA']
['3384A513DEA3046A943CC75095912B0C', '942206CC5EE6CFEFF38C303A94F7AADD', '650DD1BCD2CAF0A4BCA50A3A92F34F53', '6E246C4042A03B3C1BCF84FA2DAE71BA', '12EFE62D5FAE0241129C5E32D8C86F62']
['3384A513DEA3046A943CC75095912B0C', '942206CC5EE6CFEFF38C303A94F7AADD', '650DD1BCD2CAF0A4BCA50A3A92F34F53', '6E246C4042A03B3C1BCF84FA2DAE71BA', '12EFE62D5FAE0241129C5E32D8C86F62', '1CC15A606C4AC50C9AC6AEB6BBE94BA9']
['3384A513DEA3046A943CC75095912B0C', '942206CC5EE6CFEFF38C303A94F7AADD', '650DD1BCD2CAF0A4BCA50A3A92F34F53', '6E246C4042A03B3C1BCF84FA2DAE71BA', '12EFE62D5FAE0241129C5E32D8C86F62', '1CC15A606C4AC50C9AC6AEB6BBE94BA9', '76A0138CAFD4415EA60EFC6402DC2902']
['3384A513DEA3046A943C

KeyboardInterrupt: 

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

# Total pages to scrape
page_to_scrape = 13628 #100000
start_page_no = 0
# Base URL
base_url = "https://phishtank.org/phish_search.php?page={page}&valid=y&Search=Search"

# Headers to mimic a browser
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
                  "(KHTML, like Gecko) Chrome/140.0.0.0 Safari/537.36"
}

# Use a single session for connection pooling
session = requests.Session()
session.headers.update(headers)

def scrape_page(page):
    """Scrape one page using session"""
    try:
        url = base_url.format(page=page)
        response = session.get(url, timeout=10)
        if response.status_code != 200:
            return []
        
        soup = BeautifulSoup(response.text, 'html.parser')
        table = soup.find('table', class_='data')
        if not table:
            return []
        
        results = []
        for row in table.find_all('tr'):
            cells = row.find_all('td')
            if len(cells) >= 2:
                phishid = cells[0].text
                url_text,added_on = cells[1].text.split('added on')
                label = cells[3].text
                results.append({'phishid':phishid, 'url': url_text, 'added_on':added_on, 'label': label})
        return results
    except Exception as e:
        print(f"Error scraping page {page}: {e}")
        return []

# Scrape in parallel with session pooling
all_results = []
max_workers = 20  # adjust based on CPU/network
x=0
with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = {executor.submit(scrape_page, page): page for page in range(start_page_no+1, page_to_scrape+1)}
    for future in tqdm(as_completed(futures), total=page_to_scrape-start_page_no+1):
        data = future.result()
        if data:
            all_results.extend(data)
            x+=1
            if x % 250 == 0:
                df = pd.DataFrame(all_results)
                df.to_csv('firstall_features_scraped_valid_phishtank_scraped_dataset0.csv', index=False)

# Convert to DataFrame and save
df = pd.DataFrame(all_results)
df.to_csv('firstall_features_scraped_valid_phishtank_scraped_dataset0.csv', index=False)
print("✅ Scraping complete!")


 33%|███▎      | 4560/13629 [09:12<2:38:33,  1.05s/it]

Error scraping page 4558: HTTPSConnectionPool(host='phishtank.org', port=443): Read timed out. (read timeout=10)
Error scraping page 4559: HTTPSConnectionPool(host='phishtank.org', port=443): Read timed out. (read timeout=10)
Error scraping page 4560: HTTPSConnectionPool(host='phishtank.org', port=443): Read timed out. (read timeout=10)


 33%|███▎      | 4561/13629 [09:12<2:11:19,  1.15it/s]

Error scraping page 4562: HTTPSConnectionPool(host='phishtank.org', port=443): Read timed out. (read timeout=10)Error scraping page 4561: HTTPSConnectionPool(host='phishtank.org', port=443): Read timed out. (read timeout=10)



 33%|███▎      | 4564/13629 [09:13<1:21:30,  1.85it/s]

...

Error scraping page 4573: HTTPSConnectionPool(host='phishtank.org', port=443): Read timed out. (read timeout=10)
Error scraping page 4574: HTTPSConnectionPool(host='phishtank.org', port=443): Read timed out. (read timeout=10)


 34%|███▎      | 4577/13629 [09:14<20:07,  7.50it/s]

Error scraping page 4576: HTTPSConnectionPool(host='phishtank.org', port=443): Read timed out. (read timeout=10)
Error scraping page 4575: HTTPSConnectionPool(host='phishtank.org', port=443): Read timed out. (read timeout=10)
Error scraping page 4577: HTTPSConnectionPool(host='phishtank.org', port=443): Read timed out. (read timeout=10)


100%|█████████▉| 13628/13629 [32:39<00:00,  6.96it/s] 


✅ Scraping complete!


In [1]:
#processeed = 13628   
valid_phish = 20580
import pandas as pd
df = pd.read_csv('all_features_scraped_valid_phishtank_scraped_dataset.csv')
df.label.value_counts()

label
VALID PHISH    405000
Name: count, dtype: int64

In [2]:
df.describe()
sum(df.duplicated())
#df.to_csv('valid_phishtank_scraped_dataset.csv')

57

In [ ]:
#merger


d1 = pd.read_csv('firstall_features_scraped_valid_phishtank_scraped_dataset0.csv')
d2 = pd.read_csv("all_features_scraped_valid_phishtank_scraped_dataset.csv")

print(d1.info())
print(d2.info())
print(sum(d1.duplicated()))
print(sum(d2.duplicated()))
merger_df = pd.concat((d1,d2), ignore_index=True)
print(merger_df.info())
print(sum(merger_df.duplicated()))

merger_df.drop_duplicates()

merger_df.to_csv(r"datasets\four_row_phishtank_scraped_dataset_670K.csv", index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 272160 entries, 0 to 272159
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   phishid   272160 non-null  int64 
 1   url       272160 non-null  object
 2   added_on  272160 non-null  object
 3   label     272160 non-null  object
dtypes: int64(1), object(3)
memory usage: 8.3+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 405000 entries, 0 to 404999
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   phishid   405000 non-null  int64 
 1   url       405000 non-null  object
 2   added_on  405000 non-null  object
 3   label     405000 non-null  object
dtypes: int64(1), object(3)
memory usage: 12.4+ MB
None
9
57
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 677160 entries, 0 to 677159
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   phi

In [16]:
x=0
print(merger_df.iloc[0])
for i in merger_df.url:
    if 'http' in i and 'https' not in i:
        x+=1
print(x)

phishid                                               9226710
url         https://docs.google.com/presentation/d/e/2PACX...
added_on                                Oct 1st 2025 12:06 AM
label                                             VALID PHISH
Name: 0, dtype: object
61732


In [39]:
import kagglehub
import os
import pandas as pd
# Download latest version
big_dataset_path = kagglehub.dataset_download("pilarpieiro/tabular-dataset-ready-for-malicious-url-detection")

print("Path to dataset files:", big_dataset_path)
train_df = pd.read_csv(os.path.join(big_dataset_path, 'train_dataset.csv'))
test_df = pd.read_csv(os.path.join(big_dataset_path, 'test_dataset.csv'))

Path to dataset files: C:\Users\rrpra\.cache\kagglehub\datasets\pilarpieiro\tabular-dataset-ready-for-malicious-url-detection\versions\2


In [5]:
train_df.columns

Index(['url', 'label', 'source', 'url_has_login', 'url_has_client',
       'url_has_server', 'url_has_admin', 'url_has_ip', 'url_isshorted',
       'url_len', 'url_entropy', 'url_hamming_1', 'url_hamming_00',
       'url_hamming_10', 'url_hamming_01', 'url_hamming_11', 'url_2bentropy',
       'url_3bentropy', 'url_count_dot', 'url_count_https', 'url_count_http',
       'url_count_perc', 'url_count_hyphen', 'url_count_www',
       'url_count_atrate', 'url_count_hash', 'url_count_semicolon',
       'url_count_underscore', 'url_count_ques', 'url_count_equal',
       'url_count_amp', 'url_count_letter', 'url_count_digit',
       'url_count_sensitive_financial_words', 'url_count_sensitive_words',
       'url_nunique_chars_ratio', 'path_len', 'path_count_no_of_dir',
       'path_count_no_of_embed', 'path_count_zero', 'path_count_pertwent',
       'path_has_any_sensitive_words', 'path_count_lower', 'path_count_upper',
       'path_count_nonascii', 'path_has_singlechardir', 'path_has_upperdir'

In [35]:

train_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6728848 entries, 0 to 6728847
Data columns (total 60 columns):
 #   Column                               Dtype  
---  ------                               -----  
 0   url                                  object 
 1   label                                int64  
 2   source                               object 
 3   url_has_login                        int64  
 4   url_has_client                       int64  
 5   url_has_server                       int64  
 6   url_has_admin                        int64  
 7   url_has_ip                           int64  
 8   url_isshorted                        int64  
 9   url_len                              int64  
 10  url_entropy                          float64
 11  url_hamming_1                        float64
 12  url_hamming_00                       float64
 13  url_hamming_10                       float64
 14  url_hamming_01                       float64
 15  url_hamming_11                  

In [ ]:
features = ['url_has_login', 'url_has_client',
       'url_has_server', 'url_has_admin', 'url_has_ip', 'url_isshorted',
       'url_len', 'url_entropy', 'url_hamming_1', 'url_hamming_00',
       'url_hamming_10', 'url_hamming_01', 'url_hamming_11', 'url_2bentropy',
       'url_3bentropy', 'url_count_dot', 'url_count_https', 'url_count_http',
       'url_count_perc', 'url_count_hyphen', 'url_count_www',
       'url_count_atrate', 'url_count_hash', 'url_count_semicolon',
       'url_count_underscore', 'url_count_ques', 'url_count_equal',
       'url_count_amp', 'url_count_letter', 'url_count_digit',
       'url_count_sensitive_financial_words', 'url_count_sensitive_words',
       'url_nunique_chars_ratio', 'path_len', 'path_count_no_of_dir',
       'path_count_no_of_embed', 'path_count_zero', 'path_count_pertwent',
       'path_has_any_sensitive_words', 'path_count_lower', 'path_count_upper',
       'path_count_nonascii', 'path_has_singlechardir', 'path_has_upperdir',
       'query_len', 'query_count_components', 'pdomain_len',
       'pdomain_count_hyphen', 'pdomain_count_atrate',
       'pdomain_count_non_alphanum', 'pdomain_count_digit', 'tld_len',
       'tld_is_sus', 'pdomain_min_distance', 'subdomain_len',
       'subdomain_count_dot']
dependent_feature = ['label']



In [37]:
from xgboost import XGBClassifier

model = XGBClassifier()
model.fit(train_df[features], train_df[dependent_feature])

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, ...)

In [44]:
import numpy as np
ypred = model.predict(test_df[features])
ytrue = np.array(test_df[dependent_feature]).ravel()   # ensure 1D
ypred = ypred.ravel()  # ensure 1D

corr = ypred == ytrue

accuracy = np.sum(corr) / len(ypred)
print("Accuracy:", accuracy)


Accuracy: 0.9346604740303398
